In [3]:
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import hist
# from hist import Hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import hist
from topcoffea.modules.histEFT import HistEFT
NanoAODSchema.warn_missing_crossrefs = False

from coffea.analysis_tools import PackedSelection
from topcoffea.modules import utils
import topcoffea.modules.eft_helper as efth

In [4]:
# histogram style
hep.style.use("CMS")
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'legend.fontsize':20}
plt.rcParams.update(params)

In [5]:
# fname = "/cms/cephfs/data/store/user/hnelson2/mc/RunIISummer16NanoAODv6/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/15F67CF8-4FA2-424C-ADA6-4A1065BDBB1D.root"
# fname = "/cms/cephfs/data/store/user/hnelson2/mc/NanoGen/tW_yukawa_redo/NanoGen_tW_yukawa/nanoGen_115.root"
# fname = "/cms/cephfs/data/store/user/hnelson2/mc/central_ttbar_nanoAOD/UL17/nanoAOD_TTto2L2Nu_1Jets_smeft_MTT_0to700/NAOD-00000_1632.root"
fname = "/cms/cephfs/data/store/user/hnelson2/mc/NanoGen/tWLO_SMEFTsimtop/NanoGen_tWLO_rwgt1/nanoGen_376.root"

In [6]:
# # Load in events from root file
# events = NanoEventsFactory.from_root(
#     fname,
#     schemaclass=NanoAODSchema.v6,
#     metadata={"dataset": "TLNuto2L2Nu"},
# ).events()

# Load in events from root file
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TLNuto2L2Nu"},
).events()

In [8]:
events.LHEPart.fields

['pt', 'eta', 'phi', 'mass', 'incomingpz', 'pdgId', 'status', 'spin']

In [9]:
lhe0 = events.LHEPart[0]

In [11]:
lhe1 = events.LHEPart[1]
lhe2 = events.LHEPart[2]

In [25]:
for i in lhe0: 
    print(lhe0[i].status)

[-1, -1, 1, 1, 1, 1, 1]
[-1, -1, 1, 1, 1, 1, 1]
[-1, -1, 1, 1, 1, 1, 1]
[-1, -1, 1, 1, 1, 1, 1]
[-1, -1, 1, 1, 1, 1, 1]
[-1, -1, 1, 1, 1, 1, 1]
[-1, -1, 1, 1, 1, 1, 1]


In [42]:
for i in range(20):
    print(events.LHEPart[i].pdgId)

[-5, 21, 13, -14, -5, -15, 16]
[21, -5, 13, -14, -5, -11, 12]
[-5, 21, 11, -12, -5, -13, 14]
[-5, 21, 11, -12, -5, -15, 16]
[-5, 21, 13, -14, -5, -15, 16]
[-5, 21, 15, -16, -5, -13, 14]
[21, -5, 15, -16, -5, -15, 16]
[21, -5, 13, -14, -5, -13, 14]
[21, 5, -11, 12, 5, 15, -16]
[5, 21, -11, 12, 5, 11, -12]
[-5, 21, 13, -14, -5, -13, 14]
[-5, 21, 15, -16, -5, -13, 14]
[5, 21, -11, 12, 5, 15, -16]
[5, 21, -11, 12, 5, 13, -14]
[5, 21, -15, 16, 5, 11, -12]
[-5, 21, 11, -12, -5, -13, 14]
[-5, 21, 13, -14, -5, -11, 12]
[5, 21, -13, 14, 5, 15, -16]
[21, 5, -15, 16, 5, 11, -12]
[-5, 21, 11, -12, -5, -11, 12]


In [55]:
IDs=events.LHEPart[:,4].pdgId
check=IDs[abs(IDs)!=5]
np.shape(check)

<Array [] type='0 * int32[parameters={"__doc__": "PDG ID of LHE particles"}]'>

In [49]:
test = np.array([5, 5, 5, 4, 5])
check = test[test!=5]
np.shape(check)

(1,)

In [21]:
np.shape(lhepart[0][1])

()

In [6]:
# dir(events.PSWeight)
events.PSWeight.__doc__

'PS weights (w_var / w_nominal);   [0] is ISR=2 FSR=1; [1] is ISR=1 FSR=2[2] is ISR=0.5 FSR=1; [3] is ISR=1 FSR=0.5;'

In [7]:
events.LHEScaleWeight.__doc__

'LHE scale variation weights (w_var / w_nominal); [0] is MUF="0.5" MUR="0.5"; [1] is MUF="1.0" MUR="0.5"; [2] is MUF="2.0" MUR="0.5"; [3] is MUF="0.5" MUR="1.0"; [4] is MUF="2.0" MUR="1.0"; [5] is MUF="0.5" MUR="2.0"; [6] is MUF="1.0" MUR="2.0"; [7] is MUF="2.0" MUR="2.0"'

In [35]:
import topcoffea.modules.corrections as tc_cor

In [36]:
tc_cor.AttachScaleWeights(events)

In [42]:
events.renormUp_factUp

<Array [0.901, 0.969, 0.968, ... 0.943, 0.972] type='16000 * float64'>

In [3]:
# fname = '/project01/ndcms/hnelson2/ttbarEFT/nanoGen/TT01j2l_S1/nanoGen_101.root'
# fname = '/afs/crc.nd.edu/user/h/hnelson2/ttbarEFT/input_samples/nanoGen/djr_test_nanoGen.root'
# fname = '/cms/cephfs/data/store/user/hnelson2/tW_EFT/nanoGen/test_v2/NanoGen_tW_OGstpt/nanoGen_101.root'

# fname = '/cms/cephfs/data/store/user/hnelson2/tW_EFT/nanoGen/test_v4/NanoGen_tW_test/nanoGen_3.root'
# fname = '/cms/cephfs/data/store/user/hnelson2/tW_EFT/nanoGen/test_v5/NanoGen_tW_OGstpt/nanoGen_11.root'
# fname = '/cms/cephfs/data/store/user/hnelson2/mc/test/Jan24/NanoGen_TT01j2l_alt_stpt/nanoGen_1.root'
# fname = "/cms/cephfs/data/store/user/hnelson2/noEFT/nanoGen/tW/NanoGen_tW_noEFT/nanoGen_101.root"

fname = '/cms/cephfs/data/store/user/hnelson2/tW_EFT/nanoGen/fixed_rwgts/NanoGen_tW_OGstpt/nanoGen_365.root'

In [4]:
# Load in events from root file
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TLNuto2L2Nu"},
).events()

In [17]:
wc_lst = utils.get_list_of_wc_names(fname)
print(wc_lst)

['cHtbRe', 'ctGRe', 'ctGIm', 'cHQ3', 'cbWRe', 'cQl3', 'cleQt3Re', 'ctWRe', 'cleQt1Re']


In [5]:
print(len(events))

16000


In [6]:
print(events["genWeight"])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [7]:
eft_coeffs = events['EFTfitCoefficients']

1.0578204978628492e-12


In [16]:
print(len(eft_coeffs[0]))

55


In [9]:
print(eft_coeffs[1])

[0.00611, 4.86e-14, 1.72e-13, -0.000221, ... -8.25e-14, 2.95e-15, 1.9e-13, -9.43e-15]


In [10]:
print(eft_coeffs[2])

[0.00601, 8.22e-13, 2.89e-13, -0.000126, ... -1.36e-14, -6.28e-14, -3.11e-15]


In [11]:
# Initialize objects
genpart = events.GenPart
is_final_mask = genpart.hasFlags(["fromHardProcess","isLastCopy"])
ele  = genpart[is_final_mask & (abs(genpart.pdgId) == 11)]
mu   = genpart[is_final_mask & (abs(genpart.pdgId) == 13)]

In [12]:
######## Lep selection  ########
e_selec = ((ele.pt>20) & (abs(ele.eta)<2.5))
m_selec = ((mu.pt>20) & (abs(mu.eta)<2.5))
leps = ak.concatenate([ele[e_selec],mu[m_selec]],axis=1)
l0 = leps[ak.argmax(leps.pt, axis=-1, keepdims=True)]

In [13]:
leps = leps[ak.argsort(leps.pt, axis=-1, ascending=False)]

In [14]:
nleps = ak.num(leps)
# leps = leps[nleps==2]

In [18]:
proc_axis = hist.axis.StrCategory([], name="process", growth=True)
nleps_hist = HistEFT(proc_axis, hist.axis.Regular(bins=10, start=0, stop=10, name="nleps", label="nleps"), wc_names=wc_lst, label="Events")
event_weights = np.ones_like(events['event'])

In [19]:
print(type(nleps_hist))

<class 'topcoffea.modules.histEFT.HistEFT'>


In [22]:
print(len(eft_coeffs))

16000


In [24]:
nleps_hist.fill(nleps, process="tW", weight=event_weights, eft_coeff=eft_coeffs)

TypeError: fill() got multiple values for argument 'eft_coeff'

In [28]:
l0 = leps[ak.argmax(leps.pt, axis=-1, keepdims=True)]
l0 = leps[:,0]
l1 = leps[:,1]

In [29]:
dr = l0.delta_r(l1)

In [30]:
dr

<Array [2.81, 2, 2.85, ... 3.88, 3.36, 2.81] type='4754 * float32'>

In [ ]:
events.LHE.fields

In [ ]:
events.LHE.HTIncoming

In [ ]:
events.GenJet.fields

In [ ]:
events.LHEPart.fields

In [ ]:
events.GenPart.fields

In [ ]:
events.LHEWeight.fields

In [ ]:
events.LHEWeight.DJR10

In [ ]:
events.LHEWeight.DJR21

In [ ]:
events.LHEWeight.DJR32